> reverse engineering

- https://docs.google.com/presentation/d/1BkAjGqIqgomQpj6j0-oZORCeq72sH7_sOPjlflnYm_A/edit?pli=1#slide=id.p

In [1]:
import os
os.environ['http_proxy'] = 'http://127.0.0.1:7890'
os.environ['https_proxy'] = 'http://127.0.0.1:7890'

In [6]:
import transformer_lens
from transformer_lens import utils
from transformer_lens import HookedTransformer
import torch

- model cfg
    - https://transformerlensorg.github.io/TransformerLens/generated/code/transformer_lens.HookedTransformerConfig.html
- `logits, cache = mode.run_witch_cache(dataset)`
    - 运行模型时同时捕获中间计算结果（通常是某些层或模块的输出）以便后续分析和调试
    - `cache["pattern", layer, "attn"]`: 从指定的 layer 层获取注意力模块的注意力模式（权重）。
        - 形状为 [batch_size, num_heads, seq_length, seq_length] 的张量，包含注意力权重。
    - `cache["pre", layer, "mlp"]`: 从指定的 layer 层获取 MLP 模块的预激活输出。
        - 形状为 [batch_size, seq_length, d_model] 的张量，包含在激活函数应用之前的值。
    - `cache["post", layer, "mlp"]`: 从指定的 layer 层获取 MLP 模块的后激活输出。
        - 形状为 [batch_size, seq_length, d_model] 的张量，包含在激活函数应用之后的值。
- `model.run_with_hooks(..., fwd_hooks=[])`

### modules

- Embed()/Unembed()
    - Embed: W_E
    - Unembed: W_U
- MLP()
    - W_in
    - W_out
- Attention()
    - QKVO:
        - W(weights)
            - W_Q, W_K, W_V, W_O
        - b(biases)
            - b_Q, b_K, b_V, b_O

### utils

In [3]:
utils.get_act_name("post", 1)

'blocks.1.mlp.hook_post'

### tokens

- model.to_str_tokens
    - id => token
- model.to_single_token
    - string => id

In [7]:
model = HookedTransformer.from_pretrained("gelu-1l").to(torch.float32)

Loaded pretrained model gelu-1l into HookedTransformer
Changing model dtype to torch.float32


In [8]:
model.cfg.d_vocab

48262

In [9]:
model.to_str_tokens(torch.arange(10))

['<|EOS|>', '<|BOS|>', '<|PAD|>', '!', '"', '#', '$', '%', '&', "'"]

In [15]:
model.to_single_token('hello')

24684

### basic usage

In [ ]:
# Load a model (eg GPT-2 Small)
model = transformer_lens.HookedTransformer.from_pretrained("gpt2-small")

# Run the model and get logits and activations
logits, activations = model.run_with_cache("Hello World")

In [ ]:
logits.shape